In [ ]:
import numpy as np
import cvxpy as cp

class BipedalSLIPMPC:
    def __init__(self, mass, spring_constant, gravity, time_step, horizon):
        self.m = mass
        self.k = spring_constant
        self.g = gravity
        self.dt = time_step
        self.N = horizon

        # Initial state: [y, v_y, theta, omega]
        self.x0 = np.array([0.5, 0, 0, 0])  # Standing on the ground

    def update_state(self, x, torque):
        """
        Update the state of the SLIP.
        x: State vector [y, v_y, theta, omega]
        torque: Control input (torque applied by actuator at the joint)
        """
        y, v_y, theta, omega = x
        new_y = y + self.dt * v_y
        new_v_y = v_y + self.dt * (-self.k / self.m * (theta - 1) + self.g)
        new_theta = theta + self.dt * omega
        new_omega = omega + self.dt * (torque / self.m)  # Torque affects angular velocity

        return cp.hstack([new_y, new_v_y, new_theta, new_omega])

    def run_mpc(self):
        x = cp.Variable((self.N, 4))  # State variables over the horizon
        u = cp.Variable(self.N - 1)   # Control inputs over the horizon

        # Objective: Maximize the sum of heights
        objective = cp.Maximize(cp.sum(x[:, 0]))

        # Constraints
        constraints = [x[0, :] == self.x0]
        for i in range(self.N - 1):
            constraints.append(x[i+1, :] == self.update_state(x[i, :], u[i]))
            constraints.append(u[i] >= -10)  # Torque limits
            constraints.append(u[i] <= 10)

        # Define and solve the optimization problem
        prob = cp.Problem(objective, constraints)
        prob.solve()

        # Extract the optimal control sequence
        return u.value

# Example usage
bipedal_slip = BipedalSLIPMPC(mass=1.0, spring_constant=1000, gravity=9.81, time_step=0.01, horizon=20)
optimal_torque_sequence = bipedal_slip.run_mpc()
print("Optimal torque sequence:", optimal_torque_sequence)

Optimal torque sequence: [-9.99999995e+00 -9.99999992e+00 -9.99999988e+00 -9.99999984e+00
 -9.99999979e+00 -9.99999976e+00 -9.99999969e+00 -9.99999946e+00
 -9.99999904e+00 -9.99999853e+00 -9.99999819e+00 -9.99999611e+00
 -9.99999044e+00 -9.99998519e+00 -9.99992716e+00 -9.99961988e+00
  6.16119958e-04  4.15637278e-04  2.07778722e-04]
